# 4章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# ライブラリをインポートする。
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 7.7 MB/s 
     |████████████████████████████████| 282 kB 54.2 MB/s 
     |████████████████████████████████| 13.4 MB 30.8 MB/s 
     |████████████████████████████████| 2.2 MB 61.1 MB/s 
     |████████████████████████████████| 895 kB 37.5 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556722 sha256=39e48e6803b5b9241ee4c9b3c7668b8e81de09220b48126019476a1e5026229e
  Stored in directory: /Users/harukikondo/Library/Caches/pip/wheels/45/b7/f5/a21e68db846eedcd00d69e37d60bab3f68eb20b1d99cdff652
Successfully built ipadic
You should consider upgrading via the '/Users/harukikondo/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import torch
from transformers import BertJapaneseTokenizer, BertModel

In [3]:
# モデルには、東北大学の研究チームの事前学習モデルを指定する。
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# トークナイザを用意する。
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Downloading: 100%|██████████| 258k/258k [00:00<00:00, 1.06MB/s]
Downloading: 100%|██████████| 110/110 [00:00<00:00, 53.1kB/s]


In [4]:
# 4-4
tokenizer.tokenize('明日は自然言語処理の勉強をしよう。')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [5]:
# 「#」は、単語の一番最初以外のトークンに付与される。
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう。')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '。']

In [9]:
# [UNK]は、未知の単語を表すトークン
tokenizer.tokenize('機械学習を中国語にすると机器学习だ。')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '。']

In [10]:
# ファイアウォールを含んだ文章をトークン化
tokenizer.tokenize('PCI DSSの要件の一つには、ファイアウォールを設置することが含まれている。')

['PC',
 '##I',
 'DS',
 '##S',
 'の',
 '要件',
 'の',
 '一つ',
 'に',
 'は',
 '、',
 'ファイア',
 '##ウォール',
 'を',
 '設置',
 'する',
 'こと',
 'が',
 '含ま',
 'れ',
 'て',
 'いる',
 '。']

In [7]:
# トークンをさらにID化(符号化する)
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう。')
print(input_ids)

[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 8, 3]


In [12]:
# ファイアウォールを含んだ文章をID列化
input_ids2 =  tokenizer.encode('PCI DSSの要件の一つには、ファイアウォールを設置することが含まれている。')
print(input_ids2)

[2, 3794, 28645, 7126, 28583, 5, 8909, 5, 993, 7, 9, 6, 19764, 25693, 11, 783, 34, 45, 14, 1610, 20, 16, 33, 8, 3]


In [11]:
# ID列をトークン列に変換する。
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。', '[SEP]']

In [13]:
# 複数の文章を取り扱うために、トークン列の長さを統一する処理
text = '明日の天気は晴れだ。'
# ID列化
encoding = tokenizer(
    text, max_length=12, padding='max_length', truncation=True
)
print('# encoding:')
print(encoding)
# トークン化
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [14]:
# max_lengthを6に変更する。
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=True
)
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print(tokens)

['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [15]:
# 複数の文章をID列化する。
text_list = ['明日の天気は晴れだ。','パソコンが急に動かなくなった。']
tokenizer(
    text_list, max_length=10, padding='max_length', truncation=True
)

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [16]:
# 文字列が最大のものに合わせる。
tokenizer(text_list, padding='longest')

{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [17]:
# ID列化 + 数値配列をテンソル化する。
tokenizer(
    text_list,
    max_length=10,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
# モデルのロード
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)
# BERTをGPUに載せる
bert = bert.cuda() 

Downloading: 100%|██████████| 479/479 [00:00<00:00, 220kB/s]
Downloading: 100%|██████████| 445M/445M [00:06<00:00, 66.2MB/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# モデルの設定項目を出力する。
# max_position_embeddings 最大で入力できるトークン列の長さは512
# num_hidden_layers 隠れ層の数
# hidden_size BERTの出力の次元数
# モデルのパラメーター数 約1億1千万個
print(bert.config)

In [ ]:
# 今回入力する文字列
text_list = [
    '明日は自然言語処理の勉強をしよう。',
    '明日はマシーンラーニングの勉強をしよう。'
]

# 文章の符号化
encoding = tokenizer(
    text_list,
    max_length=32,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# データをGPUに載せる
encoding = { k: v.cuda() for k, v in encoding.items() } 

# BERTでの処理
# それぞれの入力は2次元のtorch.Tensor
output = bert(**encoding) 
# 最終層の出力
last_hidden_state = output.last_hidden_state 

In [ ]:
# bert(**encoding) と意味は同一
output = bert(
    input_ids=encoding['input_ids'], 
    attention_mask=encoding['attention_mask'],
    token_type_ids=encoding['token_type_ids']
)

In [ ]:
# テンソルのサイズの出力する。
# バッチサイズ(まとめて処理する文章の数)、系列長、隠れ状態の次元数
print(last_hidden_state.size()) 

In [ ]:
# BERTでの推論を行う時のコード
with torch.no_grad():
    output = bert(**encoding)
    last_hidden_state = output.last_hidden_state

In [ ]:
# CPUにうつす。
last_hidden_state = last_hidden_state.cpu() 
# numpy.ndarrayに変換
last_hidden_state = last_hidden_state.numpy() 
# リストに変換
last_hidden_state = last_hidden_state.tolist() 